In [1]:
import pandas as pd
df=pd.read_excel("tny.xlsx")
df = df.rename(columns={'winner':'w_name','loser':'l_name','tourney_year_id':'tny_id',\
                           'tourney_name_x':'tny_name','start_date':'tny_date','match_index':'match_num'})
df['match_num'] = df['match_num'].astype(str)
df['match_num'] = [s.replace('MS','') for s in df['match_num']]
df['tny_date'] = df['tny_date'].astype(str)
df['tny_date'] = [s.replace('.','') for s in df['tny_date']]

In [2]:
df['tny_name'] = ['Davis Cup' if 'Davis Cup' in s else s for s in df['tny_name']]
df['tny_name'] = [s.replace('Australian Chps.','Australian Open').replace('Australian Open-2',\
            'Australian Open').replace('U.S. National Chps.','US Open') for s in df['tny_name']]

In [3]:
import datetime
import re
def normalize_name(s,tour='atp'):
    if tour=='atp':
        s = s.replace('-',' ')
        s = s.replace('Stanislas','Stan').replace('Stan','Stanislas')
        s = s.replace('Alexandre','Alexander')
        s = s.replace('Federico Delbonis','Federico Del').replace('Federico Del','Federico Delbonis')
        s = s.replace('Mello','Melo')
        s = s.replace('Cedric','Cedrik')
        s = s.replace('Bernakis','Berankis')
        s = s.replace('Hansescu','Hanescu')
        s = s.replace('Teimuraz','Teymuraz')
        s = s.replace('Vikor','Viktor')
        s = s.rstrip()
        s = s.replace('Alex Jr.','Alex Bogomolov')
        s = s.title()
        sep = s.split(' ')
        return ' '.join(sep[:2]) if len(sep)>2 else s
    else:
        return s


TOUR='atp'
df['w_name'] = [normalize_name(x,tour=TOUR) for x in df['w_name']]
df['l_name'] = [normalize_name(x,tour=TOUR) for x in df['l_name']]
df['tny_name'] = ['Davis Cup' if 'Davis Cup' in s else s for s in df['tny_name']]
df['tny_name'] = [s.replace('Australian Chps.','Australian Open').replace('Australian Open-2',\
	            'Australian Open').replace('U.S. National Chps.','US Open') for s in df['tny_name']]

def format_match_df(df,ret_strings=[],abd_strings=[]):
    #df['pbp'] = [None]*len(df)
    df = df[~df['tny_date'].isin(['nan'])]
    #df = df.dropna(subset=['tny_date'])
    df = df.rename(columns={'winner_name':'w_name','loser_name':'l_name'})
    grand_slam_d = dict(zip(['Australian Open','Roland Garros','Wimbledon','US Open'],[1]*4))
    df['is_gs'] = [name in grand_slam_d for name in df['tny_name']]

    # Get dates into the same format
    df['tny_date'] = [datetime.datetime.strptime(str(x), "%Y%m%d").date() for x in df['tny_date']]
    df['match_year'] = [x.year for x in df['tny_date']]
    df['match_month'] = [x.month for x in df['tny_date']]
    df['score'] = [re.sub(r"[\(\[].*?[\)\]]", "", str(s)) for s in df['score']] # str(s) fixes any nans
    df['score'] = ['RET' if 'RET' in s else s for s in df['score']]
    df['w_swon'] = df['winner_first_serve_points_won'] + df['winner_second_serve_points_won'] 
    df['l_swon'] = df['loser_first_serve_points_won'] + df['loser_second_serve_points_won']
    df['w_rwon'] = df['winner_first_serve_return_won'] + df['winner_second_serve_return_won']
    df['l_rwon'] = df['loser_first_serve_return_won'] + df['loser_second_serve_return_won']
    df['w_rpt'] = df['winner_first_serve_return_total'] + df['winner_second_serve_return_total']
    df['l_rpt'] = df['loser_first_serve_return_total'] + df['loser_second_serve_return_total']

    # get rid of leading 0s in tny_id
    df['tny_id'] = ['-'.join(t.split('-')[:-1] + [t.split('-')[-1][1:]]) if t.split('-')[-1][0]=='0' else t \
                                    for t in df['tny_id']]

    # get rid of matches involving a retirement
    #df['score'] = ['ABN' if score.split(' ')[-1] in ('abandoned','ABN','ABD','DEF','def','unfinished','Walkover') \
    #                            else score for score in df['score']]
    #ret_strings = ['ABN','DEF','In Progress','RET','W/O',' RET',' W/O','nan','walkover']
    #ret_d = dict(zip(ret_strings,[1]*len(ret_strings)))
    #df = df.loc[[i for i in range(len(df)) if df['score'][i] not in ret_d]]
    df = df.sort_values(by=['tny_date','tny_name','match_num'], ascending=True).reset_index()
    del df['index']; return df

In [4]:
ret_strings = ('ABN','DEF','In Progress','RET','W/O',' RET',' W/O','nan','walkover')
abd_strings = ('abandoned','ABN','ABD','DEF','def','unfinished','Walkover')
atp_all_matches = format_match_df(df,ret_strings=ret_strings,abd_strings=abd_strings)
atp_all_matches = atp_all_matches[['tny_id','tny_name','tny_date','match_num','surface','is_gs','w_name','l_name',
                'winner_player_id','loser_player_id','tourney_round_name','match_year','match_month','score']]

In [5]:
print atp_all_matches.shape
atp_all_matches.head()

(74886, 14)


,tny_id,tny_name,tny_date,match_num,surface,is_gs,w_name,l_name,winner_player_id,loser_player_id,tourney_round_name,match_year,match_month,score
0,2001-7308,Adelaide,2001-01-01,001,Hard,False,Tommy Haas,Nicolas Massu,h355,m655,Finals,2001,1,6-3 6-1
1,2001-7308,Adelaide,2001-01-01,002,Hard,False,Tommy Haas,Jason Stoltenberg,h355,s331,Semi-Finals,2001,1,7-6 6-4
2,2001-7308,Adelaide,2001-01-01,003,Hard,False,Nicolas Massu,Tim Henman,m655,h336,Semi-Finals,2001,1,3-6 7-5 6-2
3,2001-7308,Adelaide,2001-01-01,004,Hard,False,Tommy Haas,Lleyton Hewitt,h355,h432,Quarter-Finals,2001,1,6-4 0-6 6-1
4,2001-7308,Adelaide,2001-01-01,005,Hard,False,Jason Stoltenberg,Alberto Martin,s331,m590,Quarter-Finals,2001,1,6-3 6-4


In [6]:
# tnynametest = atp_all_matches
# grandslam = 0.0
# tourfinals = 0.0
# masters = 0.0
# olympics = 0.0
# othergames = 0.0
# for i in range(len(tnynametest)):
#     if "Grand Slam" in tnynametest['tny_name'][i]:
#         grandslam = grandslam + 1.0
#     elif "Tour Finals" in tnynametest['tny_name'][i]:
#         tourfinals = tourfinals + 1.0
#     elif "Masters" in tnynametest['tny_name'][i]:
#         masters = masters + 1.0
#     elif "Olympics" in tnynametest['tny_name'][i]:
#         olympics = olympics + 1.0
#     else:
#         othergames = othergames + 1.0
        
# print grandslam, tourfinals, masters, olympics, othergames

## Split the training and test data

In [7]:
c = 2016
test_yr = 2017

train_df,test_df = atp_all_matches[atp_all_matches['match_year']<=c],atp_all_matches[atp_all_matches['match_year']==test_yr]

train_len = len(train_df)

print len(train_df)
print len(test_df)

62669
4155


In [8]:
import numpy as np
# import elo_538 as elo
import elo_538_jy as elo

# COUNT = 0
# players_list = np.union1d(train_df.w_name, train_df.l_name)
# players_elo = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))]))
# surface_elo = {}
# for surface in ('Hard','Clay','Grass'):
#     surface_elo[surface] = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))])) 

# elo_1s, elo_2s = [],[]
# surface_elo_1s, surface_elo_2s = [],[]
# elo_obj = elo.Elo_Rater()

# for i, row in train_df.iterrows():
#     surface = row['surface']; is_gs = row['is_gs']
#     # append elos, rate, update
#     w_elo,l_elo = players_elo[row['w_name']],players_elo[row['l_name']]
#     elo_1s.append(w_elo.value);elo_2s.append(l_elo.value)    
    
# #     elo_obj.rate_1vs1(w_elo,l_elo,is_gs,counts=COUNT)
#     elo_obj.rate_1vs1(w_elo,l_elo,is_gs,counts=COUNT, tny_name=row['tny_name'], tny_round_name=row['tourney_round_name'])

    
# #     surface_elo_1s.append(surface_elo[surface][row['w_name']].value if surface in ('Hard','Clay','Grass') else w_elo.value)
# #     surface_elo_2s.append(surface_elo[surface][row['l_name']].value if surface in ('Hard','Clay','Grass') else l_elo.value)
# #     if surface in ('Hard','Clay','Grass'):
# #         new_elo1, new_elo2 = elo_obj.rate_1vs1(surface_elo[surface][row['w_name']],surface_elo[surface][row['l_name']],is_gs,counts=COUNT)

# # add columns
# train_df['w_elo'], train_df['l_elo'] = elo_1s, elo_2s
# # train_df['sw_elo'], train_df['sl_elo'] = surface_elo_1s, surface_elo_2s

In [9]:
COUNT = 1
players_list = np.union1d(train_df.w_name, train_df.l_name)
players_elo = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))]))
surface_elo = {}
for surface in ('Hard','Clay','Grass'):
    surface_elo[surface] = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))])) 

elo_1s, elo_2s = [],[]
surface_elo_1s, surface_elo_2s = [],[]
elo_obj = elo.Elo_Rater()

for i, row in train_df.iterrows():
    surface = row['surface']; is_gs = row['is_gs']
    # append elos, rate, update
    w_elo,l_elo = players_elo[row['w_name']],players_elo[row['l_name']]
    elo_1s.append(w_elo.value);elo_2s.append(l_elo.value)    
    
#     elo_obj.rate_1vs1(w_elo,l_elo,is_gs,counts=COUNT)
    elo_obj.rate_1vs1(w_elo,l_elo,is_gs,counts=COUNT, tny_name=row['tny_name'], tny_round_name=row['tourney_round_name'])
    
    surface_elo_1s.append(surface_elo[surface][row['w_name']].value if surface in ('Hard','Clay','Grass') else w_elo.value)
    surface_elo_2s.append(surface_elo[surface][row['l_name']].value if surface in ('Hard','Clay','Grass') else l_elo.value)
    if surface in ('Hard','Clay','Grass'):
        new_elo1, new_elo2 = elo_obj.rate_1vs1(surface_elo[surface][row['w_name']],surface_elo[surface][row['l_name']],is_gs,counts=COUNT)

# add columns
train_df['w_elo_538'], train_df['l_elo_538'] = elo_1s, elo_2s
train_df['w_sf_elo_538'], train_df['l_sf_elo_538'] = surface_elo_1s, surface_elo_2s

/Users/jingyaxun/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jingyaxun/Library/Python/2.7/lib/python/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
# # New Testing method
# COUNT = 1
# players_list = np.union1d(atp_all_matches.w_name, atp_all_matches.l_name)
# players_elo = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))]))
# surface_elo = {}
# # for surface in ('Hard','Clay','Grass'):
# #     surface_elo[surface] = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))])) 

# elo_1s, elo_2s = [],[]
# # surface_elo_1s, surface_elo_2s = [],[]
# elo_obj = elo.Elo_Rater()
# correct_num = 0

# test_df_cpy = test_df.copy()

# for i, row in test_df.iterrows():
    
#     test_df_cpy = generate_elo_test(test_df_cpy, 0)
#     test_df_cpy = generate_elo_test(test_df_cpy, 1)
    
    
#     ###############
#     surface = row['surface']; is_gs = row['is_gs']
#     # append elos, rate, update
#     w_elo,l_elo = players_elo[row['w_name']],players_elo[row['l_name']]
#     # print("w_elo: {}, l_elo: {}, i: {}".format(w_elo.value, l_elo.value, i))
#     if (w_elo.value > l_elo.value):
#         correct_num += 1
#     elo_1s.append(w_elo.value);elo_2s.append(l_elo.value)    
    
#     elo_obj.rate_1vs1(w_elo,l_elo,is_gs,counts=COUNT, tny_name=row['tny_name'], tny_round_name=row['tourney_round_name'])

    
#     # surface_elo_1s.append(surface_elo[surface][row['w_name']].value if surface in ('Hard','Clay','Grass') else w_elo.value)
#     # surface_elo_2s.append(surface_elo[surface][row['l_name']].value if surface in ('Hard','Clay','Grass') else l_elo.value)
#     # if surface in ('Hard','Clay','Grass'):
#     #     new_elo1, new_elo2 = elo_obj.rate_1vs1(surface_elo[surface][row['w_name']],surface_elo[surface][row['l_name']],is_gs,counts=COUNT)

# # add columns
# atp_all_matches['w_elo_538'], atp_all_matches['l_elo_538'] = elo_1s, elo_2s
# # atp_all_matches['w_sf_elo_538'], atp_all_matches['l_sf_elo_538'] = surface_elo_1s, surface_elo_2s

# print(float(correct_num)/16384)

In [12]:
# elo_538 rankings

rankings = sorted(zip(players_elo.keys(),players_elo.values()),key=lambda x: x[1].value,reverse=True)
rankings

[(u'Novak Djokovic', elo_538_jy.Rating(2798.219, times=1340)),
 (u'Andy Murray', elo_538_jy.Rating(2668.276, times=1012)),
 (u'Roger Federer', elo_538_jy.Rating(2625.327, times=1709)),
 (u'Rafael Nadal', elo_538_jy.Rating(2523.973, times=1419)),
 (u'Milos Raonic', elo_538_jy.Rating(2405.173, times=404)),
 (u'Juan Martin', elo_538_jy.Rating(2398.629, times=481)),
 (u'Kei Nishikori', elo_538_jy.Rating(2393.243, times=530)),
 (u'Stanislas Wawrinka', elo_538_jy.Rating(2386.552, times=765)),
 (u'Robin Soderling', elo_538_jy.Rating(2365.279, times=506)),
 (u'Jo Wilfried', elo_538_jy.Rating(2357.486, times=651)),
 (u'Tomas Berdych', elo_538_jy.Rating(2350.805, times=989)),
 (u'Marin Cilic', elo_538_jy.Rating(2350.351, times=636)),
 (u'Nick Kyrgios', elo_538_jy.Rating(2324.424, times=122)),
 (u'Gael Monfils', elo_538_jy.Rating(2311.445, times=663)),
 (u'Richard Gasquet', elo_538_jy.Rating(2302.032, times=800)),
 (u'Jack Sock', elo_538_jy.Rating(2296.147, times=233)),
 (u'Dominic Thiem', elo_53

In [14]:
# elo_538_sf rankings
# rankings = sorted(zip(surface_elo.keys(),surface_elo.values()),key=lambda x: x[1].value,reverse=True)
# rankings

surface_elo

{'Clay': {u'Miles Bugby': elo_538_jy.Rating(1500.000, times=1),
  u'Sam Barry': elo_538_jy.Rating(1500.000, times=1),
  u'Marc Rosset': elo_538_jy.Rating(1743.192, times=30),
  u'Dorian Descloix': elo_538_jy.Rating(1500.000, times=1),
  u'Alejandro Mendoza': elo_538_jy.Rating(1500.000, times=1),
  u'Marcos Daniel': elo_538_jy.Rating(1867.835, times=37),
  u'Gael Particelli': elo_538_jy.Rating(1500.000, times=1),
  u'Andrei Karatchenia': elo_538_jy.Rating(1500.000, times=1),
  u'Clement Larriere': elo_538_jy.Rating(1500.000, times=1),
  u'Pavel Shchaya': elo_538_jy.Rating(1500.000, times=1),
  u'Deni Zmak': elo_538_jy.Rating(1500.000, times=1),
  u'Marcus Sarstrand': elo_538_jy.Rating(1479.597, times=2),
  u'Scott Clayton': elo_538_jy.Rating(1500.000, times=1),
  u'Maximilian Kolleritsch': elo_538_jy.Rating(1500.000, times=1),
  u'Javier Marti': elo_538_jy.Rating(1820.298, times=8),
  u'Florian Mayer': elo_538_jy.Rating(1913.540, times=148),
  u'Goncalo Andrade': elo_538_jy.Rating(1500.

In [15]:
list1 = []
list2 = []

for i in range(len(test_df)):
    name = test_df['w_name'][i + train_len]
    if name in players_elo.keys():
        if name not in list1:
            list1.append(name)
    else:
        if name not in list2:
            list2.append(name)

print len(list1)
print len(list2)
list1

# for s in list1:
#     print players_elo[s]

df_test1 = test_df

for s in list2:
    df_test1 = df_test1[df_test1.w_name != s]
    df_test1 = df_test1[df_test1.l_name != s]
    
df_test2 = df_test1
df_test2.reset_index(drop=True,inplace=True)
df_test2

sumofright = 0.0
for i in range(len(df_test2)):
    if df_test2['w_name'][i] in players_elo and df_test2['l_name'][i] in players_elo:
        if players_elo[df_test2['w_name'][i]].value - players_elo[df_test2['l_name'][i]].value > 0:
            sumofright = sumofright + 1


sumofright/len(df_test2)

319
26


0.6476377952755905

In [19]:
surfaces = surface_elo.keys()

for surface in surface_elo.keys():
    new_plyr = set()
    for i in range(len(test_df)):
        name = test_df['w_name'][i + train_len]
        if name not in surface_elo[surface].keys():
            new_plyr.add(name)


    df_test1 = test_df

    for s in new_plyr:
        df_test1 = df_test1[df_test1.w_name != s]
        df_test1 = df_test1[df_test1.l_name != s]

    df_test2 = df_test1
    df_test2.reset_index(drop=True,inplace=True)
    df_test2

    sumofright = 0.0
    for i in range(len(df_test2)):
        if df_test2['w_name'][i] in surface_elo and df_test2['l_name'][i] in surface_elo:
            if surface_elo[df_test2['w_name'][i]].value - surface_elo[df_test2['l_name'][i]].value > 0:
                sumofright = sumofright + 1

    sumofright/len(df_test2)

{u'Miles Bugby': elo_538_jy.Rating(1500.000, times=1), u'Sam Barry': elo_538_jy.Rating(1500.000, times=1), u'Marc Rosset': elo_538_jy.Rating(1714.612, times=6), u'Dorian Descloix': elo_538_jy.Rating(1500.000, times=1), u'Alejandro Mendoza': elo_538_jy.Rating(1500.000, times=1), u'Marcos Daniel': elo_538_jy.Rating(1034.725, times=6), u'Gael Particelli': elo_538_jy.Rating(1500.000, times=1), u'Andrei Karatchenia': elo_538_jy.Rating(1500.000, times=1), u'Clement Larriere': elo_538_jy.Rating(1500.000, times=1), u'Pavel Shchaya': elo_538_jy.Rating(1500.000, times=1), u'Deni Zmak': elo_538_jy.Rating(1500.000, times=1), u'Marcus Sarstrand': elo_538_jy.Rating(1500.000, times=1), u'Scott Clayton': elo_538_jy.Rating(1500.000, times=1), u'Maximilian Kolleritsch': elo_538_jy.Rating(1500.000, times=1), u'Javier Marti': elo_538_jy.Rating(1500.000, times=1), u'Florian Mayer': elo_538_jy.Rating(2002.373, times=65), u'Goncalo Andrade': elo_538_jy.Rating(1500.000, times=1), u'Marcos Ondruska': elo_538_j

ZeroDivisionError: float division by zero